Huggingface BERT

In [48]:
!pip install --upgrade tensorflow tensorflow-hub

In [49]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.17.0
Num GPUs Available:  1


In [50]:
from transformers import BertTokenizer, TFBertModel
import numpy as np
from sklearn.neighbors import NearestNeighbors
import json

# 모델과 토크나이저 로드
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [51]:
# JSON 파일 열기 (여기서 파일을 업로드한 후 사용해야 합니다)
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving naver_review_모센즈스위트 성수점_1709396208.json to naver_review_모센즈스위트 성수점_1709396208.json


In [52]:
with open(file_name, 'r', encoding='utf-8') as json_file:
    review_data = json.load(json_file)

In [53]:
# 텍스트를 임베딩으로 변환하는 함수
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] 토큰의 임베딩 반환

In [54]:
# 모든 리뷰의 "content" 임베딩 생성
embeddings = []
for review in review_data:
    content = review['content']
    embedding = get_embedding(content)
    embeddings.append(embedding)

embeddings = np.vstack(embeddings)

In [55]:
# KNN 모델 학습
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(embeddings)

# 키워드를 임베딩
keyword = "바닐라빈라떼"  # 원하는 키워드를 입력하세요
keyword_embedding = get_embedding(keyword)

# 유사도 분석
distances, indices = knn.kneighbors(keyword_embedding)

# 유사한 리뷰와 코사인 유사도 출력
print("키워드와 유사한 리뷰 및 코사인 유사도:")
for i, index in enumerate(indices[0]):
    print(f"리뷰: {review_data[index]['content']}")
    print(f"코사인 유사도: {1 - distances[0][i]:.4f}\n")

키워드와 유사한 리뷰 및 코사인 유사도:
리뷰: 굿
코사인 유사도: 1.0000

리뷰: 맛있어어요
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 맛있어요
코사인 유사도: 1.0000

리뷰: 좋아요 
코사인 유사도: 1.0000

리뷰: 좋아요
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 궁금했는데 맛있게 잘 먹었습니다 👍
코사인 유사도: 0.9321

리뷰: 여사장님그외다른직원분들도엄청친절하시고카이막으로유명한데카이막도맛있습니다. 음료도라떼진짜말씀대로묵직한게라떼맛집입니다.
코사인 유사도: 0.9182



Tensorflow BertModel

In [56]:
!pip install tensorflow_text
import tensorflow_text as text  # Registers the ops.

In [57]:
import tensorflow_hub as hub
import tensorflow_text as text

In [58]:
preprocess_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"

In [59]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [60]:
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/4"

In [61]:
bert_model = hub.KerasLayer(encoder_url)

In [62]:
# 텍스트를 임베딩으로 변환하는 함수
def get_embedding(text):
    preprocessed_text = bert_preprocess_model([text])
    embeddings = bert_model(preprocessed_text)['pooled_output']
    return embeddings.numpy()

In [63]:
# 모든 리뷰의 "content" 임베딩 생성
embeddings = []
for review in review_data:
    content = review['content']
    embedding = get_embedding(content)
    embeddings.append(embedding)

embeddings = np.vstack(embeddings)

In [64]:
# KNN 모델 학습
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(embeddings)

# 키워드를 임베딩
keyword = "세련됐어요"  # 원하는 키워드를 입력하세요
keyword_embedding = get_embedding(keyword)

# 유사도 분석
distances, indices = knn.kneighbors(keyword_embedding)

# 유사한 리뷰와 코사인 유사도 출력
print("키워드와 유사한 리뷰 및 코사인 유사도:")
for i, index in enumerate(indices[0]):
    print(f"리뷰: {review_data[index]['content']}")
    print(f"코사인 유사도: {1 - distances[0][i]:.4f}\n")

키워드와 유사한 리뷰 및 코사인 유사도:
리뷰: 굿
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 좋아요 
코사인 유사도: 1.0000

리뷰: 맛있어어요
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 굿
코사인 유사도: 1.0000

리뷰: 좋아요
코사인 유사도: 1.0000

리뷰: 맛있어요
코사인 유사도: 1.0000

리뷰: 특이하고 좋아요 분위기좋고 맛도있어요
코사인 유사도: 0.9936

리뷰: 카이막 맛있었어요👍
코사인 유사도: 0.9924

